---
title: "Natural Language Processing"
# author: "Harlow Malloc"
date: "2022-03-29"
categories: [NLP, transfer learning, code]
image: "NLP.png"
format:
  html:
    code-fold: true
    fig-cap-location: bottom
    tbl-cap-location: top
    toc: true
    toc-depth: 3
    toc-location: left
    toc-title: "Contents"
crossref:
  fig-title: "Figure"
  eq-title: "Equation"
  tbl-title: "Table"
jupyter: python3
execute:
  cache: true
  freeze: auto
---

# Objectives
In this notebook, we are going to deep dive into natural language processing (NLP) using Deep Learning ([info](https://medium.com/dair-ai/deep-learning-for-nlp-an-overview-of-recent-trends-d0d8f40a776d)). Relying on the pretrained language model, we are going to fine-tune it to classify the reviews, and it works as sentiment analysis. 

Based on a `language model` which has been trained to guess what the next word in the text is, we will apply transfer learning method for this NLP task.


![Transfer learning workflow for movie classifier](https://github.com/fastai/fastbook/blob/master/images/att_00027.png?raw=1){#fig-transfer-learning fig-cap="Transfer learning workflow for movie classifier."}

As shown in @fig-transfer-learning, we will start with the Wikipedia language model with a subset which we called ``Wikitext103``. Then, we are going to create an IMDb language model which predicts the next word of a movie reviews. This intermediate learning will help us to learn about IMDb-specific kinds of words like the name of actors and directors. Afterward, we end up with our classifier. 

::: {.callout-tip}
## Three-Step Transfer Learning Process
1. **Pre-trained Model**: Start with Wikitext103 language model
2. **Domain Adaptation**: Fine-tune on IMDb movie reviews  
3. **Task-Specific**: Create sentiment classifier
:::


# Text Preprocessing

In order to build a language model with many complexities such as different sentence lengths in long documents, we can build a neural network model to deal with that issue. We apprehended that categorical variables (words) can be used as independent variables for a neural network (using embedding matrix).  Then, we could do the same thing with text.

First, we concatenate all the documents in our dataset into a big long string and split it into words. Our independent variables will be the sequence of words starting with the first word and ending with the second last, and our dependent variable would be the sequence of words starting with the second word and ending with the last words. 

In our vocab, it might exist the very common words and new words. For new words, because we don't have any pre-knowledge, so we will just initialize the corresponding row with a random vector. 

These above steps can be listed as below:
- Tokenization: convert the text into a list of words
- Numericalization: make a list of all the unique words which appear, and convert each word into a number, by looking up its index in the vocab.
- Language model data loader creation : handle creating dependant variables
- Language model creation: handle input list by using recurrent neural network.

## Tokenization
Basically, tokenization convert the text into list of words. Firstly, we will grab our IMDb dataset and try out the tokenizer with all the text files.

In [ ]:
#| echo: true
#| output: false
#| eval: true
#| warning: false
from fastai.text.all import *
path = untar_data(URLs.IMDB)
# path.ls()

In [ ]:
#| echo: true
#| output: true
#| eval: true
#| warning: false
files = get_text_files(path,folders=['train','test','unsup'])

The default English word tokenizer that FastAI used is called `SpaCy` which uses a sophisticated riles engine for particular words and URLs. Rather than directly using ```SpacyTokenizer```, we are going to use ```WordTokenizer``` which always points to fastai's current default word tokenizer. 

In [ ]:
#| echo: true
#| output: true
#| eval: true
#| warning: false
txt = files[0].open().read()
txt[:60]
spacy = WordTokenizer()
toks = first(spacy([txt]))

print(coll_repr(toks,30))

### Sub-word tokenization
In additions to word tokenizer, sub-word tokenizer is really useful for languages which the spaces are not necessary for separations of components in a sentence (e.g: Chinese). To handle this, we will do 2 steps:
- Analyze a corpus of documents to find the most commonly occurring groups of letters which form the vocab
- Tokenize the corpus using this vocab of sub-word units

For example, we will first look into 2000 movie reviews:

In [ ]:
#| echo: true
#| output: true
#| eval: true
#| warning: false
txts = L(o.open().read() for o in files[:2000])
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

Then, the long underscore is when we replace the space and we can know where the sentences actually start and stop. 

In [ ]:
#| echo: true
#| output: true
#| eval: true
#| warning: false
subword(10000)

If we use a larger vocab, then most common English words will end up in the vocab themselves, and we will not need as many to represent a sentence. So, there is a compromise to take into account when choosing sub-word vocab: A larger vocab means more fewer tokens per sentence which means faster training, less memory, less state for the model to remember, but it comes to the downside of larger embedding matrix and requiring more data to learn.

## Numericalization
In order to numericalize the tokens, we need to call ```setup``` first to create the vocab. 

In [ ]:
#| echo: true
#| output: true
#| eval: true
#| warning: false
tkn = Tokenizer(spacy)
toks300 = txts[:300].map(tkn)
toks300[0]
num = Numericalize()
num.setup(toks300)
coll_repr(num.vocab,20)

The results return our rule tokens first, and it is followed by word appearances, in frequency order.
Once we created our Numerical object, we can use it as if it were a function.

In [ ]:
nums = num(toks)[:20]
nums

In [ ]:
' '.join(num.vocab[o] for o in nums)

Now, we have already had numerical data, we need to put them in batches for our model.

### Batches of texts
Recalling the batch creation for the images when we have to reshape all the images to be same size before grouping them together in a single tensor for the efficient calculation purposes. It is a little bit different when dealing with texts because it is not desirable to resize the text length. Also, we want the model read texts in order so that it can efficiently predict what the next word is. This suggests that each new batch should begin precisely where the previous one left off.

So, the text stream will be cut into a certain number of batches (with batch size) with preserving the order of the tokens. Because we want the model to read continuous rows of the text.

To recap, at every epoch, we shuffle our collection of documents and concatenate them into a stream of tokens. Then, that stream will be cut into a batch of fixed size consecutive mini stream. The model will read these mini streams in order and it will produce the same activation.


In [ ]:
nums300 = toks300.map(num)
dl = LMDataLoader(nums300)
x,y = first(dl)
x.shape, y.shape

The batch size is 64x72. 64 is the default batch size and 72 is the default sequence length.

# Training a Text Classifier
## Create a language model using DataBlock
By default, `fastai` handles tokenization and numericalization automatically when `TextBlock` is passed to `DataBlock`. 

In [ ]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
blocks=TextBlock.from_folder(path, is_lm=True),
get_items=get_imdb, splitter=RandomSplitter(0.1)).dataloaders(path, path=path, bs=128, seq_len=80)

## Fine-tuning the language model

In this step, we are going to create a learner which is going to learn and predict the next word of a `movie review`. It will take the data from data loader, pretrained model (``AWD_LSTM``), apply dropout technique and take accuracy as well as perplexity metrics into account. Particularly, `accuracy` metric is used to evaluate how the correctness when the model tries to predict the next word, while `perplexity` metric is used to track the (exponential) value of cross-entropy loss.

In [ ]:
learn = language_model_learner(
dls_lm, AWD_LSTM, drop_mult=0.3,
metrics=[accuracy, Perplexity()]).to_fp16()

Then, we will perform intermediate model training by fitting the model in one training cycle. 

In [ ]:
#| echo: true
#| output: true
#| eval: true
#| warning: false
learn.fit_one_cycle(1,2e-2)

After few minutes of training, we got the prediction accuracy which is around 29-30 percent.
In order to reuse the pre-trained model, we can easily save the model with PyTorch. In this case, we are going to save only learnable parameters (i.e., weight and bias of a model via `state_dict`) and the updated parameters after one epoch training will be stored at `learn.path/'models'/'one_epoch_training_torch.pth'`.

In [ ]:
# Option 1: Save with FastAI
# learn.save('one_epoch_training')

# Option 2: Save with PyTorch
import torch
model_save_path = learn.path/'models'/'one_epoch_training_torch.pth'
torch.save(learn.model.state_dict(), model_save_path)
# print(f"Model saved to: {model_save_path}")

Once the trainable parameters are stored, we can later load those parameter to the compatible model for further training 

::: {.callout-important}
## Implementation Note: PyTorch Model Loading
When using `torch.load()`, be cautious about the `weights_only` parameter. For security reasons, consider using `weights_only=True` when loading models from untrusted sources to prevent execution of arbitrary code.
:::

In [ ]:
# Option 1: Use FastAI's load method
# learn.load('one_epoch_training', strict=False)

# Option 2: Use PyTorch to load the saved model
import torch
model_load_path = learn.path/'models'/'one_epoch_training_torch.pth'
state_dict = torch.load(model_load_path, weights_only=False)
learn.model.load_state_dict(state_dict, strict=False)
# print(f"Model loaded from: {model_load_path}")

After loading the pre-saved model, we can unfreeze it and train it for few more epochs. Then, let's see the improvement of the accuracy.

In [ ]:
#| warning: false
#| eval: true
learn.unfreeze()

learn.fit_one_cycle(10,2e-3)

As we can see from the training process, the accuracy has improved progressively. At the end of ten cycle training, the accuracy has increased to around 35 percent. 
To perform model finetuning, we save the model parameters except the last activation function layer. To do that, we can save it with `save_encoder`

In [ ]:
learn.save_encoder('finetuned')

In this step, we have fine tuned the language model. Now, we will fine tune this language model using the IMDb sentiment labels.

Although the model is pre-designed for next word prediction, we can also use this model to generate texts. For example, we can self-create a sentence with some words and we parses this sentence to the model to generate a new sentence which has one word longer than the parsed sentence. Leveraging this capability, we are going to create 40 new words from that randomized content.

In [ ]:
#| echo: true
#| output: true
#| eval: true
#| warning: false

TEXT = "I liked this movie so"

N_WORDS = 40

N_SENTENCES = 2

preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)]

Let's see the generation of new inventing words

In [ ]:
print("\n".join(preds))

## Fine-tuning the classification model

Previously, we built a language model to predict the next word of a document given the input text. Now, we are going to move to the classifier which predicts the sentiment of a document.


In [ ]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In the `TextBlock.from_folder()` function, we do not set `is_lm=True` because we tell `TextBlock` that we had regular labelled data rather than using next word as a label as we did for prediction.

::: {.callout-important}
## Create training batch for sentiment classification
It is important to say that  we need to collate all the items in a batch into a single tensor, and a tensor has a fixed size. Therefore, we need to pad/crop/squish our sequences to make the inputs have the same length. 
For the characteristics of the input, we will apply padding here so that each batch contains the documents with similar sizes, which is the largest size of the document in that batch. (every batch may not have similar sizes):
- At first, we sort documents by length prior to each epoch.
- we use special padding token to expand the shortest texts to the same length as the target size in a batch.
:::

## Fine-tuning the classification model
so far, we have `finetuned` encoder, which stores the trained parameter weights from previous step. Now, we are going to create a learner that load the `finetuned` encoder for fine-tuning. Then, we are going to fine-tune it over several epoch.

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5,
                                metrics=accuracy).to_fp16()

In [ ]:
learn.load_encoder('finetuned')

As we are training a classification task, we only need to unfreeze several last layers of the encoder instead of unfreezing all the layers.

In [ ]:
#| warning: false
#| eval: true
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

We can also further  unfreeze more layers and do training to see if the accuracy improves or not.

In [ ]:
#| warning: false
#| eval: true
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
#| warning: false
#| eval: true
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))